In [1]:
from selenium import webdriver

from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions

from bs4 import BeautifulSoup
from collections import defaultdict

import time
import os

import numpy as np
import pandas as pd

In [2]:
def set_ff_preferences():
    profile = webdriver.FirefoxProfile()
    profile.set_preference('browser.download.folderList', 2) # custom location
    profile.set_preference('browser.download.dir', (os.getcwd()+'/data/'))
    profile.set_preference('browser.download.manager.showWhenStarting', False)
    profile.set_preference('browser.helperApps.neverAsk.saveToDisk', "application/csv, text/csv")
    return profile

def create_empty_ff_driver():
    profile = set_ff_preferences()
    a_driver = webdriver.Firefox(firefox_profile=profile)
    return a_driver

def create_driver_opening_url():
    a_driver = create_empty_ff_driver()
    return a_driver

def soup_from_driver(a_driver):
    soup = BeautifulSoup(a_driver.page_source, 'html.parser')
    return soup

def clean_from_text(text):
#     cleaned_text = text.replace(",","").strip()
    cleaned_text = text.strip()
    return cleaned_text

In [3]:
def init_driver(url):
    a_driver = create_driver_opening_url()
    a_driver.get(url)
    return a_driver
    
def dl_data(a_driver):
    dl_button = WebDriverWait(a_driver, 10).until(
    expected_conditions.presence_of_element_located(
        (By.XPATH, """//*[@id="download-and-save"]""")))
    dl_button.click()
    
def get_csv(url):
    a_driver = init_driver(url)
    dl_data(a_driver)
    a_driver.close()
    

In [4]:
def get_csv_file_str():
    return 'data/'+str(os.listdir('data')[0])

def csv_to_df(file_str):
    df=pd.read_csv(file_str)
    df.rename(columns={"URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)":"URL"},inplace=True)
    return df

In [5]:
def clean_data_list(data_list):
    clean_list = ['Today','Price', 'Date','Public Records','Show Less', 'Continue reading']
    partial_list = ['Stellar', 'Sold for', '+']
    
    for i in data_list:
        if i in clean_list:
            data_list.remove(i)
    for i in data_list:
        for j in partial_list:
            if j in i:
                data_list.remove(i)
    for i in data_list:
        if i[3:5] == ', ':
            data_list.remove(i)
    for i in data_list:
        if len(i) > 30:
            data_list.remove(i)
            
#     for i in data_list:
#         if i in clean_list:
#             data_list.remove(i)
#         elif i[3:5] == ', ':
#             data_list.remove(i)
#         else:
#             for j in partial_list:
#                 if j in i:
#                     data_list.remove(i)
                    
    return data_list

In [6]:
def df_from_long_list(long_list):
    td_list = []
    for i in range(len(long_list))[::3]:
        try:
            od_list = [long_list[i],long_list[i+1],long_list[i+2]]
            td_list.append(od_list)
        except:
            None
    df = pd.DataFrame(td_list, columns =['Date', 'Event', 'Price'])
    return df

In [7]:
def clean_price(string):
    s = string
    try:
        if type(s) is str:
            s = s.split()[0]
            clean_list = ['$',',']
            for i in clean_list:
                if i in s:
                    s = s.replace(i,'')
            if len(s) == 1:
                return np.nan
        return float(s)
    except:
        return string

In [8]:
def click_expand_property_history(a_driver):
        button_xpath = "//*[contains(text(), 'See all property history')]"

        expand_button = WebDriverWait(a_driver, 10).until(
            expected_conditions.presence_of_element_located(
            (By.XPATH, button_xpath)))

        expand_button.click()

In [9]:
def scrape_data_from_expandable(a_driver):
    section_id = 'propertyHistory-expandable-segment'

    price_history_table = WebDriverWait(a_driver, 10).until(
        expected_conditions.presence_of_element_located(
            (By.ID, section_id)))
    
    return price_history_table.text.split('\n')

In [10]:
def scrape_data_from_static(a_driver):
    section_id = 'property-history-transition-node'

    price_history_table = WebDriverWait(a_driver, 10).until(
        expected_conditions.presence_of_element_located(
            (By.ID, section_id)))
    
    return price_history_table.text.split('\n')

In [11]:
def get_raw_property_history_data(a_driver):
    try:
        click_expand_property_history(a_driver)
        data = scrape_data_from_expandable(a_driver)
    except:
        data = scrape_data_from_static(a_driver)
    return data

In [12]:
def clean_df(df):
    df = df[df['Price'] != 'Continue reading']
    df['Price'] = df['Price'].apply(clean_price)
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    return df

In [13]:
def get_property_history(a_url, a_driver):
    a_driver.get(a_url)
    raw_data = get_raw_property_history_data(a_driver)
    df = df_from_long_list(clean_data_list(raw_data))
    df = clean_df(df)
    df['URL'] = a_url
    return df

In [14]:
def debug_property_history(a_url, a_driver):
    a_driver.get(a_url)
    raw_data = get_raw_property_history_data(a_driver)
    return raw_data

In [15]:
def get_properties_overview_df(url):
    get_csv(url)
    df = csv_to_df(get_csv_file_str())
    return df

In [16]:
rf_search = 'https://www.redfin.com/city/16164/FL/St-Petersburg/filter/property-type=multifamily,max-price=700k,include=forsale+mlsfsbo+construction+fsbo+foreclosed+sold-3mo,viewport=27.91387:27.72085:-82.54595:-82.86657,no-outline'
all_props_df = get_properties_overview_df(rf_search)

In [17]:
a_driver = create_driver_opening_url()
prop_hist_df = pd.DataFrame(columns=['Date','Event','Price','URL'])
for i in range(len(all_props_df))[:5]:
    url = all_props_df['URL'][i]
    fdf = get_property_history(url, a_driver)
    print(i)
    prop_hist_df = pd.concat([prop_hist_df,fdf],ignore_index=True)
a_driver.close()

0
1
2
3
4


In [18]:
prop_hist_df

,Date,Event,Price,URL
0,2021-06-22,Listed (Active),250000,http://www.redfin.com/FL/Largo/Undisclosed-add...
1,1988-07-22,Sold (Public Records),60000,http://www.redfin.com/FL/Largo/Undisclosed-add...
2,1984-03-01,Sold (Public Records),44000,http://www.redfin.com/FL/Largo/Undisclosed-add...
3,2021-06-22,Price Changed,599900,http://www.redfin.com/FL/Madeira-Beach/14000-M...
4,2021-06-18,Listed (Active),645000,http://www.redfin.com/FL/Madeira-Beach/14000-M...
5,2017-08-28,Sold (Public Records),275000,http://www.redfin.com/FL/Madeira-Beach/14000-M...
6,2017-08-25,Sold (MLS) (Sold),275000,http://www.redfin.com/FL/Madeira-Beach/14000-M...
7,2017-07-17,Pending,NaN,http://www.redfin.com/FL/Madeira-Beach/14000-M...
8,2017-07-12,Listed (Active),295000,http://www.redfin.com/FL/Madeira-Beach/14000-M...
9,NaT,"Jul 12, 2017",Delisted (Withdrawn),http://www.redfin.com/FL/Madeira-Beach/14000-M...


In [19]:
problem_url = all_props_df['URL'][1]
"""
debugging starts here
"""

'http://www.redfin.com/FL/Madeira-Beach/14000-Miramar-Ave-33708/home/47980945'

In [20]:
a_driver = create_driver_opening_url()
t = debug_property_history(all_props_df['URL'][1], a_driver)

In [21]:
print(t[17:])

['Public Records', '$275,000 (3.9%/yr)', 'Price', 'Aug, 2017', 'Aug 25, 2017', 'Date', 'Sold (MLS) (Sold)', 'Stellar MLS #U7825832', '$275,000', 'Price', 'Jul 17, 2017', 'Date', 'Pending', 'Stellar MLS #U7825832', '—', 'Price', 'Jul 12, 2017', 'Date', 'Listed (Active)', 'Stellar MLS #U7825832', '$295,000', 'Price', '+3', 'Listing provided courtesy of My Florida Regional MLS DBA Stellar MLS', "Lots of room in this uniquely laid out two family home, steps from the sands of Madeira Beach. Rental demand is at an all time high, especially when your with in walking distance to the beaches and local beach community activities. Opportunity awaits you if you want to live in one unit and rent the other. First floor has three bedrooms, kitchen, separate dinning room and up stairs large one bedroom, large living room and kitchen/dinning room combo. Pinellas beach trolly and all the advantages of Tampa Bay, entertainment, dinning, boating near by. You don't want to miss this one!", 'Continue readin

In [22]:
c = clean_data_list(t[17:20])

In [23]:
c

['$275,000 (3.9%/yr)']

In [ ]:
"""
I think the program skips over the one because it removed one 
before and that messes with the loop. I need to figure out how 
to cleanly fix it.
"""

In [24]:
get_property_history(all_props_df['URL'][1], a_driver)

,Date,Event,Price,URL
0,2021-06-22,Price Changed,599900,http://www.redfin.com/FL/Madeira-Beach/14000-M...
1,2021-06-18,Listed (Active),645000,http://www.redfin.com/FL/Madeira-Beach/14000-M...
2,2017-08-28,Sold (Public Records),275000,http://www.redfin.com/FL/Madeira-Beach/14000-M...
3,2017-08-25,Sold (MLS) (Sold),275000,http://www.redfin.com/FL/Madeira-Beach/14000-M...
4,2017-07-17,Pending,NaN,http://www.redfin.com/FL/Madeira-Beach/14000-M...
5,2017-07-12,Listed (Active),295000,http://www.redfin.com/FL/Madeira-Beach/14000-M...
6,NaT,"Jul 12, 2017",Delisted (Withdrawn),http://www.redfin.com/FL/Madeira-Beach/14000-M...
7,NaT,"Jul 11, 2017",Listed (Active),http://www.redfin.com/FL/Madeira-Beach/14000-M...
8,NaT,"May 27, 2014",Delisted (Withdrawn),http://www.redfin.com/FL/Madeira-Beach/14000-M...
9,NaT,"Apr 7, 2014",Relisted (Active),http://www.redfin.com/FL/Madeira-Beach/14000-M...
